In [1]:
import sim2bids
import panel as pn
from sim2bids.sim2bids import MainArea
from sim2bids.app import app
pn.extension('tabulator', 'ace', 'jsoneditor', 'ipywidgets', sizing_mode='stretch_width', notifications=True)

import warnings
warnings.filterwarnings('ignore')


NotificationArea(sizing_mode='stretch_width')

In [2]:
app.SoftwareVersion = 2.6
app.SoftwareRepository = 'https://github.com/the-virtual-brain/tvb-root/releases/tag/2.6'
app.SoftwareName = 'TVB'


In [3]:
# local version
app = MainArea()
pn.serve(app.view())


INFO:bokeh.server.server:Starting Bokeh server version 2.4.2 (running on Tornado 6.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:58236


INFO:tornado.access:200 GET / (127.0.0.1) 1174.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/jsoneditor/npm/jsoneditor@9.1.9/dist/jsoneditor.min.css (127.0.0.1) 608.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/dataframe.css (127.0.0.1) 10.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/datatabulator/tabulator-tables@4.9.3/dist/css/tabulator_simple.min.css (127.0.0.1) 22.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/npm/notyf@3/notyf.min.css (127.0.0.1) 37.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/ajax/libs/font-awesome/5.15.1/css/all.min.css (127.0.0.1) 80.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/alerts.css (127.0.0.1) 100.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/card.css (127.0.0.1) 111.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/debugger.css (127.0.0.1) 11.99ms
INFO:tornado.access:200 GET /stat

Removed all files...


Removed all files...


INFO:tornado.access:200 GET /static/extensions/panel/bundled/jsoneditor/npm/jsoneditor@9.1.9/dist/img/jsoneditor-icons.svg (127.0.0.1) 7.00ms


In [3]:
# inline version (smth you'll use on a cluster)

app = MainArea()
app.view().servable()


MaterialTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [notifications] NotificationArea(sizing_mode='stretch_width')
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [1190555018336] Tabs(dynamic=True, sizing_mode='stretch_width')
        [0] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] TextInput(name='Insert Path', sizing_mode='stretch_width')
            [2] CrossSelector(options=['.ipynb_checkpoints', ...], sizing_mode='stretch_width')
            [3] Row(margin=(50, 0, 0, 0), sizing_mode='stretch_width')
                [0] Param(MainArea, parameters=['gen_struct'], show_name=False, sizing_mode='stretch_width', widgets={'gen_struct': {...})
                [1] Param(MainArea, parameters=['gen_btn'], show_name=False, sizing_mode='stretch_width', widgets={'gen_btn': {'button_type'...})
            [4] StaticText(margin=(5, 0, 50, 20), sizing_mode='stretch_width')
        [1] WidgetBox()
        [2] Column(sizing_mode='stretch_width')
            [0] RadioButtonGroup(button_type='primary', options=['JSON files', ...])
            [1] WidgetBox(sizing_mode='stretch_width')
                [0] Markdown(str, sizing_mode='stretch_width')
                [1] Select()
        [3] Column(height=600, scroll=True, sizing_mode='stretch_width')
            [0] RadioButtonGroup(name='User Guide', options=['App 101', 'Preprocess da...], value='App 101')
            [1] StaticText(sizing_mode='stretch_width', value='<br>\nUsing the app was c...)
    [1190555019872] Markdown(str, sizing_mode='stretch_width')
    [1190555020976] Markdown(str, sizing_mode='stretch_width')
    [1190553446720] TextInput(margin=(-20, 10, 0, 10), value='inputs')
    [1190555021168] Markdown(str, sizing_mode='stretch_width')
    [1190553447248] TextInput(margin=(-20, 10, 0, 10), value='output')
    [1190555020928] Markdown(str, sizing_mode='stretch_width')
    [1190553447680] TextInput(margin=(-20, 10, 0, 10), max_length=30, value='default')
    [1190555177408] Markdown(str, sizing_mode='stretch_width')
    [1190553447344] CheckBoxGroup(margin=(-20, 10, 0, 10), options=['Traverse subfolders', ...], value=['Traverse subfolders', ...])
    [1190555177600] Markdown(str, sizing_mode='stretch_width')
    [1190553447728] Column
        [0] Markdown(str)
        [1] TextInput(margin=(-10, 0, 0, 0), value='cspeed')
    [1190553448304] Column
        [0] Markdown(str)
        [1] TextInput(margin=(-10, 0, 0, 0), value='csf')